In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import tensorflow as tf
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score
import time
assert(tf.__version__.startswith('2.3'))


from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10


In [2]:
from ampligraph.latent_features import ScoringBasedEmbeddingModel

In [3]:
# load the dataset
full_dataset = load_fb15k_237()

In [4]:
# this would go into separate classes, like we had in ampligraph 1 (for loss functions and initializers)
# initializer
def xavier(in_shape, out_shape):
    std = np.sqrt(2 / (in_shape + out_shape))
    return np.random.normal(0, std, size=(in_shape, out_shape)).astype(np.float32)

#loss function
def nll(scores_pred, eta):
    scores_neg = scores_pred[1]
    scores_pos = scores_pred[0]

    scores_neg_reshaped = tf.reshape(scores_neg, [eta, tf.shape(scores_pos)[0]])
    neg_exp = tf.exp(scores_neg_reshaped)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.reduce_sum(tf.math.log(softmax_score))
    return loss

In [5]:
rel_to_idx, ent_to_idx = create_mappings(full_dataset['train'])
train_dataset = to_idx(full_dataset['train'], rel_to_idx=rel_to_idx, ent_to_idx=ent_to_idx)

pd.DataFrame(train_dataset, columns=['s', 'p', 'o']).to_csv('fb15k_237_train.csv', 
                                                           header=False,
                                                           index=False, sep='\t')

In [6]:
test_dataset = to_idx(full_dataset['test'], rel_to_idx=rel_to_idx, ent_to_idx=ent_to_idx)

pd.DataFrame(test_dataset, columns=['s', 'p', 'o']).to_csv('fb15k_237_test.csv', 
                                                           header=False,
                                                           index=False, sep='\t')


x_filter = np.concatenate([full_dataset['train'], full_dataset['valid'], full_dataset['test']], 0)

filter_dataset = to_idx(x_filter, rel_to_idx=rel_to_idx, ent_to_idx=ent_to_idx)

pd.DataFrame(filter_dataset, columns=['s', 'p', 'o']).to_csv('fb15k_237_filter.csv', 
                                                           header=False,
                                                           index=False, sep='\t')


In [7]:
model = ScoringBasedEmbeddingModel(eta=5, 
                                                     k=100, 
                                                     max_ent_size=14505, 
                                                     max_rel_size=237,
                                                     scoring_type='TransE')


Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalN

In [8]:
# uncomment if you have already saved the checkpoint
# model.load_weights('./chkpt1')

In [9]:
model.compile(optimizer='adam', loss=nll)

Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0


In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

In [11]:
model.fit('/home/spai/code/ampligraph_projects/tf2/AmpliGraph-Lab/fb15k_237_train.csv',
         batch_size=30000,
         epochs=10, callbacks=[checkpoint])

TypeError: __init__() got an unexpected keyword argument 'epochs'

In [12]:
model.summary()

Model: "scoring_based_embedding_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
TransE (TransE)              multiple                  0         
_________________________________________________________________
corruption_generation_layer_ multiple                  0         
_________________________________________________________________
embedding_lookup_layer (Embe multiple                  1474200   
_________________________________________________________________
loss (Mean)                  multiple                  2         
Total params: 1,474,202
Trainable params: 1,474,200
Non-trainable params: 2
_________________________________________________________________


In [13]:
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/tf2/AmpliGraph-Lab/fb15k_237_test.csv')
end = time.time()

The mappings will be created.
Mappings updated with: 10317 ents, 10317 rev_ents, 224 rels and 224 rev_rels
Executing op Pack in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
      1/Unknown - 0s 27us/stepExecuting op StridedSlice in device /job:localhos

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
     20/Unknown - 0s 9ms/stepExecuting op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localho

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
     39/Unknown - 0s 8ms/stepExecuting op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in dev

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
     59/Unknown - 0s 8ms/stepExecuting op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localho

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
     78/Unknown - 1s 8ms/stepExecuting op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localho

...

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GP

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference__get_ranks_13615 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GP

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in

/home/spai/code/ampligraph_projects/tf2/AmpliGraph-Lab/ampligraph/datasets/graph_data_loader.py:158: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  yield [self.ent_emb[triples[:, 0]],


IndexError: too many indices for array

In [15]:
print(len(ranks), end-start)
print(mrr_score(ranks))


20437 4.493479490280151
0.0910748906430413


# Ignore the below code

In [12]:
# this class is responsible for training the embeddings on multiple GPUs
class ModelTrainer():
        
    def __init__(self, batch_size, max_ents, max_rels, k, dataset, seed=0):
        # max_ent_size - is the max embeddings that can be loaded in memory
        np.random.seed(seed)
        self.eta = 10
        self.models = None # would be a list later (one per device)
        self.optimizers = None # would be a list later (one per device)
        
        
        self.batch_size=batch_size
        self.num_ents = max_ents
        self.num_rels = max_rels
        self.max_ent_size = max_ents
        
        # create the dataset handle
        self.data_loader = GraphDataLoader(dataset, 
                              batch_size=batch_size, dataset_type="train")
        # create the model and optimizer
        with tf.device('GPU:0'):
            self.optimizers = tf.optimizers.Adam(lr=0.001)
            self.models = ScoringBasedEmbeddingModel(eta=self.eta, 
                                                     k=k, 
                                                     max_ent_size=self.max_ent_size, 
                                                     max_rel_size=self.num_rels,
                                                     scoring_type='ComplEx')
        
        self.batch_size=batch_size
        
        self.k = self.models.k
        
    
    @tf.function(experimental_relax_shapes=True)
    def train_step(self, inputs, optimizer):
        with tf.GradientTape() as tape:
            # get the model predictions
            preds = self.models(inputs, training=0)
            # compute the loss
            loss = nll(preds, self.eta)
            # regularizer - will be in a separate class like ampligraph 1
            loss += (0.0001 * (tf.reduce_sum(tf.pow(tf.abs(self.models.encoding_layer.ent_emb), 3)) + \
                              tf.reduce_sum(tf.pow(tf.abs(self.models.encoding_layer.rel_emb), 3))))

        # compute the grads
        gradients = tape.gradient(loss, [self.models.encoding_layer.ent_emb, 
                                         self.models.encoding_layer.rel_emb])
        # update the trainable params
        optimizer.apply_gradients(zip(gradients, [self.models.encoding_layer.ent_emb, 
                                                  self.models.encoding_layer.rel_emb]))   
        return loss
        
                

    def train(self, epochs = 5):
        # create the generator 
        dataset = tf.data.Dataset.from_generator(self.data_loader.get_batch,
                                             output_types=(tf.int32),
                                             output_shapes=((None, 3)))
        dataset = dataset.prefetch(0)



        for i in range(epochs):
            total_loss = []
            print(i)
            self.global_epoch = i
            # train on batches
            for j, inputs in dataset.enumerate():
                self.global_batch = 0
                with tf.device('{}'.format('GPU:0')):
                    loss = self.train_step(inputs, self.optimizers)
                
                total_loss.append(loss/inputs.shape[0])
            
            print('\n\n\n\nloss------------------{}:{}'.format(i, np.mean(total_loss)))
        print('done')
        return

In [13]:

model_trainer = ModelTrainer(batch_size = 30000, 
                             max_ents = 14505, # ideally we should get from the dataset apis/partition manager
                             max_rels=237,
                             k=300,  
                             dataset='/home/spai/code/ampligraph_projects/tf2/AmpliGraph-Lab/fb15k_237_train.csv')

start = time.time()
model_trainer.train(30)
end = time.time()

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalN

/home/spai/.conda/envs/tf_nightly/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replic

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_7503 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0




loss------------------2:2.3957266807556152
3
Executing op Ran

Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replic

Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_7503 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0




loss------------------7:2.2356677055358887
8
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDat

Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replic

Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_7503 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0




loss------------------12:1.221268892288208
13
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset i

Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op IteratorGetNext in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_step_6618 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0


KeyboardInterrupt: 

In [ ]:
class ModelEvaluator():
    def __init__(self, ent_emb, rel_emb, models, max_ent_size, dataset, filterset):
        self.ent_emb = ent_emb
        self.rel_emb = rel_emb
        self.models = models
        self.max_ent_size = max_ent_size
        self.dataset_handle = GraphDataLoader(dataset, 
                                  batch_size=100, dataset_type="test")
        
    def get_next_batch(self):
        for out in self.dataset_handle:

            yield self.ent_emb[out[:, 0]], self.rel_emb[out[:, 1]], self.ent_emb[out[:, 2]]
            
    def evaluate(self):
        dataset = tf.data.Dataset.from_generator(self.get_next_batch,
                                                 output_types=(tf.float32, tf.float32, tf.float32),
                                                 output_shapes=((None, self.models.k), 
                                                                (None, self.models.k), 
                                                                (None, self.models.k)))
        dataset = dataset.prefetch(1)

        self.all_ranks = []
        for i, inputs in dataset.enumerate():
            batch_size = self.max_ent_size
            batch_count = np.int32(np.round(self.ent_emb.shape[0]/batch_size))
            overall_rank = np.zeros((inputs[0].shape[0], 2))
            for j in range(batch_count):
                ent_embs = self.ent_emb[j * batch_size : (j+1) * batch_size, :]
                rel_embs = self.rel_emb
                with tf.device('GPU:0'):
                    sub_rank, obj_rank = self.models.get_ranks(inputs, ent_embs)
                overall_rank[:, 0] +=  sub_rank.numpy()
                overall_rank[:, 1] +=  obj_rank.numpy()
            overall_rank = overall_rank + 1
            self.all_ranks.extend(overall_rank.tolist())

In [ ]:
model_evaluator = ModelEvaluator(model_trainer.models.encoding_layer.ent_emb.numpy(),
                                      model_trainer.models.encoding_layer.rel_emb.numpy(),
                                      model_trainer.models,
                                      max_ent_size=14505,
                                      dataset='/home/spai/code/ampligraph_projects/tf2/AmpliGraph-Lab/fb15k_237_test.csv',
                                      filterset='/home/spai/code/ampligraph_projects/tf2/AmpliGraph-Lab/fb15k_237_filter.csv')



start=time.time()
model_evaluator.evaluate()
print((time.time()-start)/60)

In [ ]:
mrr_score(model_evaluator.all_ranks), hits_at_n_score(model_evaluator.all_ranks, 10)

# Ignore this - partition related

In [ ]:
# Partitioning based

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')


tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8000),
    tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8000)])

logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

logical_gpus  

In [ ]:
# this class is responsible for training the embeddings on multiple GPUs
class ModelTrainer():
    
    # this class manages the partitions
    # This would later be responsible for persisting the input data and creating the partitions
    # during training time, it will load the partitions and related embeddings 
    class PartitionManager():
        def __init__(self, num_buckets, k, dataset, strategy, num_devices=1):
            self.num_buckets = num_buckets
            
            num_entities = len(set(dataset[:, 0]).union(dataset[:, 2]))
            num_rels = len(set(dataset[:, 1]))
            
            self.rel_to_idx, self.ent_to_idx = create_mappings(dataset)
            dataset = to_idx(dataset, self.ent_to_idx, self.rel_to_idx)
            
            partitioner = PARTITION_ALGO_REGISTRY.get(strategy)(dataset, k=num_buckets)
            self.partitions = partitioner.split()
            
            if num_buckets > 1:
                self.num_partitions = np.sum(np.arange(1, num_buckets+1))
            else:
                self.num_partitions = num_buckets
            
            # needs to go into the database
            self.entity_embeddings = xavier(num_entities, k)
            self.rel_embeddings = xavier(num_rels, k)
            
            self.num_devices = 1
            
            # use multiple GPUs only if we do multiple partitions
            if self.num_partitions > 1:
                self.num_devices = num_devices
            
        
        def get_next_partition(self):
            # get the next partition to train on, along with the embeddings of nodes in that partition
            for i in range(len(self.partitions)):
                
                    entities_of_partition = set(self.partitions[i][:, 0]).union(self.partitions[i][:, 2])
                    partition_dict = dict(zip(entities_of_partition, 
                                              np.arange(len(entities_of_partition))))
                    new_rel_dict = dict(zip(np.arange(len(self.rel_to_idx)), np.arange(len(self.rel_to_idx))))
                    # remap the triples to reflect the position in the embedding matrix
                    remapped_triples = to_idx(self.partitions[i], partition_dict, new_rel_dict)
                    yield partition_dict, \
                            new_rel_dict, \
                            remapped_triples, \
                            self.entity_embeddings[list(partition_dict.keys()), :], \
                            self.rel_embeddings[list(new_rel_dict.keys()), :]
        
        
        
    def __init__(self, batch_size, max_ent_size, k, dataset, strategy, num_buckets=1, num_devices=1, seed=0):
        # max_ent_size - is the max embeddings that can be loaded in memory
        np.random.seed(seed)
        self.eta = 10
        self.models = None # would be a list later (one per device)
        self.optimizers = None # would be a list later (one per device)
        self.max_ent_size = max_ent_size
        
        self.batch_size=batch_size
        self.num_devices = num_devices
        self.num_ents = len(set(dataset[:, 0]).union(dataset[:, 2]))
        self.num_rels = len(set(dataset[:, 1]))
        
        # create embedding model and optimizer
        for i in range(num_devices):
            with tf.device('GPU:{}'.format(i)):
                self.optimizers = tf.optimizers.Adam(lr=0.001)
                self.models = ScoringBasedEmbeddingModel(eta=self.eta, 
                                                         k=k, 
                                                         max_ent_size=max_ent_size, 
                                                         max_rel_size=self.num_rels,
                                                         scoring_type='ComplEx')
        
        self.batch_size=batch_size
        
        self.k = self.models.k
        self.partition_manager = self.PartitionManager(num_buckets, self.k, dataset, strategy, num_devices)
        
        # this is the hyperparams of the optimizer - would be later moved 
        # to database calls and retrieved from partition manager
        self.optimizer_hyperparams_ent = np.zeros(shape=(self.num_ents, 2, self.k), 
                                                  dtype=np.float32)
        self.optimizer_hyperparams_rel = np.zeros(shape=(self.num_rels, 2, self.k), 
                                                  dtype=np.float32)
        
        
    def train_dataset_generator(self, dataset):
        # generator for the training data
        batch_count = dataset.shape[0]//self.batch_size + 1
        for j in range(batch_count):
            inputs = dataset[j * self.batch_size : (j+1) * self.batch_size, :].astype(np.int32)
            yield inputs
            
    def update_partion_embeddings_after_train(self):
        # before changing the partition, save the trained embeddings and optimizer params
        
        self.partition_manager.entity_embeddings[list(self.ent_dict.keys()), :] = self.models.encoding_layer.ent_emb.numpy()[:len(self.ent_dict), :]
        self.partition_manager.rel_embeddings[list(self.rel_dict.keys()), :] = self.models.encoding_layer.rel_emb.numpy()[:len(self.rel_dict), :]
        
        opt_weights = self.optimizers.get_weights()
        if len(opt_weights)>0:
            self.optimizer_hyperparams_rel[list(self.rel_dict.keys()), :, :] = np.concatenate([opt_weights[2][:len(self.rel_dict)][:, np.newaxis, :], 
                                                                                                 opt_weights[4][:len(self.rel_dict)][:, np.newaxis, :]], 1)
            
            self.optimizer_hyperparams_ent[list(self.ent_dict.keys()), :, :] = np.concatenate([opt_weights[1][:len(self.ent_dict)][:, np.newaxis, :], 
                                                                                             opt_weights[3][:len(self.ent_dict)][:, np.newaxis, :]], 1)
        
        
    def change_partition(self):
        # load a new partition and update the trainable params and optimizer hyperparams
        self.ent_dict, self.rel_dict, remapped_triples, ent_embs, rel_embs = next(self.partition_iterator)
        print('partition has {} triples', remapped_triples.shape)
        self.partition_dataset_iterator = iter(self.train_dataset_generator(remapped_triples))
        self.models.partition_change_updates(len(self.ent_dict), ent_embs, rel_embs)
        if self.global_epoch >1:
            # needs to be better handled
            optimizer_rel_weights_updates_beta1 = self.optimizer_hyperparams_rel[list(self.rel_dict.keys()), 0, :]
            optimizer_rel_weights_updates_beta2 = self.optimizer_hyperparams_rel[list(self.rel_dict.keys()), 1, :]
            optimizer_ent_weights_updates_beta1 = self.optimizer_hyperparams_ent[list(self.ent_dict.keys()), 0, :]
            optimizer_ent_weights_updates_beta2 = self.optimizer_hyperparams_ent[list(self.ent_dict.keys()), 1, :]
            
            optimizer_rel_weights_updates_beta1 = np.pad(optimizer_rel_weights_updates_beta1, 
                                                         ((0, self.num_rels - optimizer_rel_weights_updates_beta1.shape[0]), 
                                                          (0,0)), 
                                                         'constant', 
                                                         constant_values=(0))
            optimizer_rel_weights_updates_beta2 = np.pad(optimizer_rel_weights_updates_beta2, 
                                                         ((0, self.num_rels - optimizer_rel_weights_updates_beta2.shape[0]), 
                                                          (0,0)), 
                                                         'constant', 
                                                         constant_values=(0))
            optimizer_ent_weights_updates_beta1 = np.pad(optimizer_ent_weights_updates_beta1, 
                                                         ((0, self.max_ent_size - optimizer_ent_weights_updates_beta1.shape[0]), 
                                                          (0,0)), 
                                                         'constant', 
                                                         constant_values=(0))
            optimizer_ent_weights_updates_beta2 = np.pad(optimizer_ent_weights_updates_beta2, 
                                                         ((0, self.max_ent_size - optimizer_ent_weights_updates_beta2.shape[0]), 
                                                          (0,0)), 
                                                         'constant', 
                                                         constant_values=(0))
            
            self.optimizers.set_weights(self.optimizers.get_weights())

            self.optimizers.set_weights([self.optimizers.iterations.numpy(), 
                                         optimizer_ent_weights_updates_beta1,
                                         optimizer_rel_weights_updates_beta1,
                                         optimizer_ent_weights_updates_beta2,
                                         optimizer_rel_weights_updates_beta2
                                        ])

            
    def get_next_batch(self):
        try:
            self.partition_iterator = iter(self.partition_manager.get_next_partition())
            # get new partition
            self.change_partition()
            while True:
                try:
                    # get batches from the current partition
                    out = next(self.partition_dataset_iterator)
                    yield out
                except StopIteration:
                    # if no more batch data - save the trained params and load next partition
                    self.update_partion_embeddings_after_train()
                    self.change_partition()
        except StopIteration:
            self.update_partion_embeddings_after_train()
            #if no more partitions, end
            return
                
    
    @tf.function(experimental_relax_shapes=True)
    def train_step(self, inputs, optimizer):
        with tf.GradientTape() as tape:
            # get the model predictions
            preds = self.models(inputs, training=0)
            # compute the loss
            loss = nll(preds, self.eta)
            # regularizer - will be in a separate class like ampligraph 1
            loss += (0.0001 * (tf.reduce_sum(tf.pow(tf.abs(self.models.encoding_layer.ent_emb), 3)) + \
                              tf.reduce_sum(tf.pow(tf.abs(self.models.encoding_layer.rel_emb), 3))))

        # compute the grads
        gradients = tape.gradient(loss, [self.models.encoding_layer.ent_emb, 
                                         self.models.encoding_layer.rel_emb])
        # update the trainable params
        optimizer.apply_gradients(zip(gradients, [self.models.encoding_layer.ent_emb, 
                                                  self.models.encoding_layer.rel_emb]))   
        return loss
        
                

    def train(self, epochs = 5):
        dataset = tf.data.Dataset.from_generator(self.get_next_batch,
                                             output_types=(tf.int32),
                                             output_shapes=((None, 3)))
        dataset = dataset.prefetch(0)



        for i in range(epochs):
            total_loss = []
            print(i)
            self.global_epoch = i

            for j, inputs in dataset.enumerate():
                self.global_batch = 0
                with tf.device('{}'.format('GPU:0')):
                    loss = self.train_step(inputs, self.optimizers)
                
                total_loss.append(loss/inputs.shape[0])
            
            print('\n\n\n\nloss------------------{}:{}'.format(i, np.mean(total_loss)))
        print('done')
        return


In [ ]:
num_buckets = 3
model_trainer = ModelTrainer(30000, 
                             14505, 
                             k=300,
                             dataset=full_dataset['train'], 
                             strategy='RandomVertices',
                             num_buckets=num_buckets, 
                             num_devices=1)

start = time.time()
model_trainer.train(1)
end = time.time()

print(end - start)

In [ ]:
# continue training for 10 more epochs
start = time.time()
model_trainer.train(1)
end = time.time()

In [ ]:
# compare loss of prev model with model trained for 20 epochs.

model_trainer = ModelTrainer(30000, 
                             14505, 
                             k=300,
                             dataset=full_dataset['train'], 
                             strategy='Bucket',
                             num_buckets=1, 
                             num_devices=1)

start = time.time()
model_trainer.train(2)
end = time.time()

print(end - start)

In [ ]:
from functools import partial



from ampligraph.evaluation import filter_unseen_entities

from ampligraph.datasets import NumpyDatasetAdapter
filter_triples = np.concatenate([full_dataset['train'],
                                full_dataset['valid'],
                                full_dataset['test']], 0).astype(np.str)

test_dataset = full_dataset['test']
dataset_handle = NumpyDatasetAdapter()
dataset_handle.use_mappings(model_trainer.partition_manager.rel_to_idx, 
                            model_trainer.partition_manager.ent_to_idx)

# need to fix filter_unseen_entities - this is temp workaround
model_trainer.models.ent_to_idx = model_trainer.partition_manager.ent_to_idx
model_trainer.models.rel_to_idx = model_trainer.partition_manager.rel_to_idx
test_dataset = filter_unseen_entities(test_dataset.astype(np.str), model_trainer.models, verbose=True)
print(test_dataset.shape)
dataset_handle.set_data(test_dataset, 'test')

print(filter_triples.shape)
filter_triples = filter_unseen_entities(filter_triples, model_trainer.models, verbose=True)
dataset_handle.set_filter(filter_triples)

print(filter_triples.shape)

class ModelEvaluator():
    def __init__(self, ent_emb, rel_emb, models, max_ent_size, dataset_handle):
        self.ent_emb = ent_emb
        self.rel_emb = rel_emb
        self.models = models
        self.max_ent_size = max_ent_size
        self.dataset_handle = dataset_handle
        
    def get_next_batch(self):
        batch_size = 100
        batch_count = np.int32(np.round(self.dataset_handle.get_size('test')/batch_size))
        test_generator = partial(self.dataset_handle.get_next_batch,
                             dataset_type='test',
                             use_filter=False, 
                             batches_count=batch_count)

        batch_iterator = iter(test_generator())
        print('Btahces: ', np.int32(np.round(batch_count/batch_size)))
        for i in range(batch_count):
            out = next(batch_iterator)

            yield self.ent_emb[out[:, 0]], self.rel_emb[out[:, 1]], self.ent_emb[out[:, 2]]
            
    def evaluate(self):
        dataset = tf.data.Dataset.from_generator(self.get_next_batch,
                                                 output_types=(tf.float32, tf.float32, tf.float32),
                                                 output_shapes=((None, self.models.k), 
                                                                (None, self.models.k), 
                                                                (None, self.models.k)))
        dataset = dataset.prefetch(1)

        self.all_ranks = []
        for i, inputs in dataset.enumerate():
            batch_size = self.max_ent_size
            batch_count = np.int32(np.round(self.ent_emb.shape[0]/batch_size))
            overall_rank = np.zeros((inputs[0].shape[0], 2))
            for j in range(batch_count):
                ent_embs = self.ent_emb[j * batch_size : (j+1) * batch_size, :]
                rel_embs = self.rel_emb
                with tf.device('GPU:0'):
                    sub_rank, obj_rank = self.models.get_ranks(inputs, ent_embs)
                overall_rank[:, 0] +=  sub_rank.numpy()
                overall_rank[:, 1] +=  obj_rank.numpy()
            overall_rank = overall_rank + 1
            self.all_ranks.extend(overall_rank.tolist())
    

In [ ]:
model_evaluator = ModelEvaluator(model_trainer.partition_manager.entity_embeddings,
                                      model_trainer.partition_manager.rel_embeddings,
                                      model_trainer.models,
                                      max_ent_size=max_entities,
                                      dataset_handle=dataset_handle)



start=time.time()
model_evaluator.evaluate()
print(i, (time.time()-start)/60)


In [ ]:
model_evaluator.all_ranks = np.array(model_evaluator.all_ranks) + 1

In [ ]:
mrr_score(model_evaluator.all_ranks), hits_at_n_score(model_evaluator.all_ranks, 2)